# Spotify recommendation system 

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('./data_folder/data.csv')
genre_data = pd.read_csv('./data_folder/data_w_genres.csv')
year_data = pd.read_csv('./data_folder/data_by_year.csv')
artist_data = pd.read_csv('./data_folder/data_by_artist.csv')

In [14]:
data.head(2),genre_data.head(2),year_data.head(2),artist_data.head(2)

(   valence  year  acousticness  \
 0   0.0594  1921         0.982   
 1   0.9630  1921         0.732   
 
                                              artists  danceability  \
 0  ['Sergei Rachmaninoff', 'James Levine', 'Berli...         0.279   
 1                                     ['Dennis Day']         0.819   
 
    duration_ms  energy  explicit                      id  instrumentalness  \
 0       831667   0.211         0  4BJqT0PrAfrxzMOxytFOIz             0.878   
 1       180533   0.341         0  7xPhfUan2yNtyFG0cUWkt8             0.000   
 
    key  liveness  loudness  mode  \
 0   10     0.665   -20.096     1   
 1    7     0.160   -12.441     1   
 
                                                 name  popularity release_date  \
 0  Piano Concerto No. 3 in D Minor, Op. 30: III. ...           4         1921   
 1                            Clancy Lowered the Boom           5         1921   
 
    speechiness   tempo  
 0       0.0366  80.954  
 1       0.4150  60.936  ,

In [13]:
data.info(),genre_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170653 entries, 0 to 170652
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   valence           170653 non-null  float64
 1   year              170653 non-null  int64  
 2   acousticness      170653 non-null  float64
 3   artists           170653 non-null  object 
 4   danceability      170653 non-null  float64
 5   duration_ms       170653 non-null  int64  
 6   energy            170653 non-null  float64
 7   explicit          170653 non-null  int64  
 8   id                170653 non-null  object 
 9   instrumentalness  170653 non-null  float64
 10  key               170653 non-null  int64  
 11  liveness          170653 non-null  float64
 12  loudness          170653 non-null  float64
 13  mode              170653 non-null  int64  
 14  name              170653 non-null  object 
 15  popularity        170653 non-null  int64  
 16  release_date      17

(None, None)

In [15]:
# create a decade column using apply and lambda
data["decade"]= data["year"].apply(lambda x: str(x)[:3]+'0s')